Data Cleaning - Aggregated Airbnb Calendar Data

# Introduction

In the following notebook, I will be cleaning an aggregation of Airbnb's calendar data of the San Francisco area. This data consists of data from 1/2018 through 12/2019.

* The aggregation source code [here](https://github.com/KishenSharma6/Airbnb-Analysis/blob/master/Project%20Codes/01.%20Raw%20Data%20Aggregation%20Scripts/2020_0129_Airbnb_Raw_Data_Aggregation.ipynb)

* Raw data [here](https://github.com/KishenSharma6/Airbnb-Analysis/tree/master/Data/01_Raw/SF%20Airbnb%20Raw%20Data)

## Read in libraries,  read in data, and set notebook preferences

**Read in libraries**

In [19]:
#Read in libraries
import swifter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

**Read in Data**

In [20]:
#Set path to get aggregated calendar data
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Air BnB - SF\Data\01_Raw\SF Airbnb Raw Data\SF Airbnb Raw Data - Aggregated\01_04_2020_Calendar_Raw_Aggregated.csv'

#Set date columns for parsing
parse_dates = ['date']

#Read in calendar data
calendar = pd.read_csv(path, sep = ',', dtype = {"listing_id" : "object"}, parse_dates=parse_dates,index_col=0, low_memory=False)

C:\Users\kishe\Miniconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


**Settings for Notebook**

In [21]:
#Set Pandas options
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows',500)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

#Set plot style
plt.style.use('ggplot')

## Preview Data

In [22]:
#Preview calendar data
display(calendar.head())

,adjusted_price,available,date,listing_id,maximum_nights,minimum_nights,price
0,NaN,t,2019-04-05,21190709,nan,nan,"$5,000.00"
1,NaN,t,2019-04-04,21190709,nan,nan,"$5,000.00"
2,NaN,t,2019-04-03,21190709,nan,nan,"$5,000.00"
3,NaN,t,2019-04-02,21190709,nan,nan,"$5,000.00"
4,NaN,t,2019-04-01,21190709,nan,nan,"$5,000.00"


In [23]:
#Print shape and dtypes of calendar data
print('Calendar data shape:',calendar.shape)
print('\nCalendar data types: \n',calendar.dtypes)

Calendar data shape: (26178939, 7)

Calendar data types: 
 adjusted_price            object
available                 object
date              datetime64[ns]
listing_id                object
maximum_nights           float64
minimum_nights           float64
price                     object
dtype: object


# Data Cleaning

## Drop columns not needed for Time Series Analysis

In [24]:
#Drop columns not needed for time series analysis
calendar.drop(columns = ['adjusted_price','maximum_nights', 'minimum_nights'], inplace = True)

#View
display(calendar)

,available,date,listing_id,price
0,t,2019-04-05,21190709,"$5,000.00"
1,t,2019-04-04,21190709,"$5,000.00"
2,t,2019-04-03,21190709,"$5,000.00"
3,t,2019-04-02,21190709,"$5,000.00"
4,t,2019-04-01,21190709,"$5,000.00"
...,...,...,...,...
61385197,f,2020-09-06,38533186,$255.00
61385198,f,2020-09-07,38533186,$342.00
61385199,f,2020-09-08,38533186,$455.00
61385200,f,2020-09-09,38533186,$522.00


## Filter dates

In [25]:
#Keep dates in 2018 and 2019
calendar = calendar.loc[(calendar['date'] > '2017-12-31') & (calendar['date'] < '2020')]

#View updated calendar shape
print('Updated calendar shape:', calendar.shape)

Updated calendar shape: (18739278, 4)


## Column cleaning and data type correction

**Price column**

In [26]:
#Remove $ and , from price 
calendar['price']=calendar['price'].replace('[,$]','', regex=True)

#Convert string to numeric
calendar['price'] =calendar['price'].swifter.apply(pd.to_numeric, errors='coerce')

**Available column**

In [27]:
#Replace 't' and 'f' in available column to True and False
calendar.available = calendar.available.swifter.apply(lambda x:  1 if x =='t' else 0)

C:\Users\kishe\Miniconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Missing Data

**Stats of missing values per column**

In [28]:
#Create function that captures missing data stats of a pandas df
def missing_stats(pandas):
    #Create empty dataframe to store missing value stats in
    missing = pd.DataFrame()
    #Capture total number and total % of missing data per column
    missing['Total'] = pandas.isnull().sum().sort_values(ascending=False)
    missing['Percent'] = (missing['Total']/pandas.isnull().count()).apply(lambda x: x * 100)
    return missing

#Check
missing_stats(calendar)

,Total,Percent
price,4613888,24.62
listing_id,0,0.00
date,0,0.00
available,0,0.00


**View rows with missing price values**

In [29]:
#View rows with missing price
calendar[calendar.price.isnull()]

,available,date,listing_id,price
1146,0,2018-11-03,6938818,nan
1147,0,2018-11-02,6938818,nan
1148,0,2018-11-01,6938818,nan
1149,0,2018-10-31,6938818,nan
1150,0,2018-10-30,6938818,nan
...,...,...,...,...
58489652,0,2019-09-03,28201824,nan
58489653,0,2019-09-04,28201824,nan
58489654,0,2019-09-05,28201824,nan
58489655,0,2019-09-06,28201824,nan


It appears that if available is false, price will be missing. Let's test that theory:

In [30]:
#Capture rows with missing price data and assign to df
missing_price = calendar[calendar['price'].isnull()]

#View 
print('Missing price values with availability: ', len(missing_price[missing_price.available == 1]))
print('Missing price values w/o availability: ', len(missing_price[missing_price.available == 0]))

Missing price values with availability:  102
Missing price values w/o availability:  4613786


Not a bad hunch :) We'll leave those values as is for our time series analysis. We will, however, drop those 102 missing values

**Drop missing values where available is true**

In [31]:
#Reset calendar index
calendar.reset_index(inplace=True)

#Get index of rows that contain missing prices in available rooms
drop = calendar.loc[(calendar['price'].isnull()) & (calendar['available'] == 1)].index.to_list()

#Remove
calendar.drop(calendar.index[drop], inplace=True)

#Check
calendar.loc[(calendar['price'].isnull()) & (calendar['available'] == 1)]

,index,available,date,listing_id,price


## Zero Price Removal

In [32]:
#Remove rows where price == 0 (most likely a data error, rent cannot be $0)
calendar = calendar[calendar['price'] > 0]

# Write file to CSV

In [33]:
#View final calendar shape
print('Final calendar shape:', calendar.shape)

Final calendar shape: (14125242, 5)


In [34]:
#Set path to write listings
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Air BnB - SF\Data\02_Intermediate\2020_0130_Calendar_Cleaned.csv'

#Write listings to path
calendar.to_csv(path, sep=',')